In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('indg_langs_orig.csv')

In [3]:
df.head()

,Geographic code,Geographic name,Global non-response rate,Data quality flag,Aboriginal languages,Mother tongue,Language spoken most often at home,Other language(s) spoken regularly at home
0,1,Canada,4.0,20000,Aboriginal languages,213225,137515,91780
1,1,Canada,4.0,20000,Algonquian languages,143160,89795,63820
2,1,Canada,4.0,20000,Blackfoot,3465,1855,3090
3,1,Canada,4.0,20000,Cree-Montagnais languages,96260,64825,37955
4,1,Canada,4.0,20000,Atikamekw,6295,5810,580


Any entry specified under Aboriginal languages as 'X languages' will be removed since these are cumulative variables for total speakers of language families.

In [4]:
df = df.loc[~df['Aboriginal languages'].str.contains('languages', case=False)]

Delete all Canada-specific entries. Just keep the provinces.

In [5]:
df = df.loc[df['Geographic name'] != "Canada"]

Reset the index for it to not give away the idea that it could be missing rows.

In [6]:
df = df.reset_index(drop=True)

Keep only the informative rows I want.

In [7]:
df = df[['Geographic name', 'Aboriginal languages', 'Mother tongue']]

Add a new column specifying Canada in each row for the Tableau map to work.

In [8]:
df.at[:,'Country'] = 'Canada'

Order the columns to in a more logical fashion.

In [9]:
df = df[['Country', 'Geographic name', 'Aboriginal languages', 'Mother tongue']]

In [10]:
df.head()

,Country,Geographic name,Aboriginal languages,Mother tongue
0,Canada,Newfoundland and Labrador,Blackfoot,0
1,Canada,Newfoundland and Labrador,Atikamekw,0
2,Canada,Newfoundland and Labrador,Montagnais (Innu),1535
3,Canada,Newfoundland and Labrador,Moose Cree,0
4,Canada,Newfoundland and Labrador,Naskapi,605


There's a bunch of whitespace in the spelling for a bunch of the aboriginal languages.

In [11]:
df['Aboriginal languages'] = df['Aboriginal languages'].str.strip()

We're gonna edit the 'n.o.s.' ('not otherwise specified') languages since the acronym is confusing.

In [12]:
#convert the language column to a list for ease
langs = list(df['Aboriginal languages'])

#function for getting rid of the n.o.s. languages
def no_nos(lang):
    
    result = lang
    
    if lang == 'Cree, n.o.s.':
        
        result = 'Cree'
        
    elif lang == 'Slavey, n.o.s.':
        
        result = 'Slavey'
        
    return result

#now the confusing n.o.s.'s are out of the way
df['Aboriginal languages'] = [no_nos(lang) for lang in langs]

In [13]:
df.head()

,Country,Geographic name,Aboriginal languages,Mother tongue
0,Canada,Newfoundland and Labrador,Blackfoot,0
1,Canada,Newfoundland and Labrador,Atikamekw,0
2,Canada,Newfoundland and Labrador,Montagnais (Innu),1535
3,Canada,Newfoundland and Labrador,Moose Cree,0
4,Canada,Newfoundland and Labrador,Naskapi,605


Find the most spoken language per `Geographic name`.

In [14]:
#create new column
df['Most spoken'] = ''

In [15]:
#how many speakers the most spoken language has
most_common_lang = df.groupby(['Geographic name']).max()['Mother tongue']

In [16]:
#fill in the new column specifying most spoken languages
for region in most_common_lang.keys():
    
    #find the most spken language for the given region
    row = df.loc[(df['Geographic name'] == region) & (df['Mother tongue'] == most_common_lang[region])]    
    row = row.reset_index(drop=True)
    most_spoken = row.loc[0,'Aboriginal languages']
    
    #fill the column 'Most spoken' for the given 'Geographic name'
    df.loc[df['Geographic name'] == region, 'Most spoken'] = most_spoken

In [17]:
df.head()

,Country,Geographic name,Aboriginal languages,Mother tongue,Most spoken
0,Canada,Newfoundland and Labrador,Blackfoot,0,Montagnais (Innu)
1,Canada,Newfoundland and Labrador,Atikamekw,0,Montagnais (Innu)
2,Canada,Newfoundland and Labrador,Montagnais (Innu),1535,Montagnais (Innu)
3,Canada,Newfoundland and Labrador,Moose Cree,0,Montagnais (Innu)
4,Canada,Newfoundland and Labrador,Naskapi,605,Montagnais (Innu)


Save the dataframe.

In [18]:
df.to_csv('indg_langs.csv', index=False)